# Loading data from an existing Experiment Pipeline

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from azureml.core import Workspace
from src.models import pipelines
ws = Workspace.from_config()

In [ ]:
from azureml.core import Experiment
experiment = Experiment(ws, 'longterm')

In [ ]:
experiment

In [ ]:
pipeline_run = None
for run in experiment.get_runs():
    if run.get_status()=='Finished':
        pipeline_run = run
        break

In [ ]:
run

In [ ]:
trip_id_step = pipeline_run.find_step_run("Add trip_id")[0]

In [ ]:
ds = pipelines.data(experiment=experiment, step_name="trip statistics", dataset_name='trip_statistics')

In [ ]:
df_statistics = ds.to_pandas_dataframe()

In [ ]:
df_statistics.head()

## Register datasets

In [ ]:
pipelines.register_output_to_workspace(experiment=experiment, step_name="trip statistics", dataset_name='trip_statistics')
pipelines.register_output_to_workspace(experiment=experiment, step_name="Add trip_id", dataset_name='data_with_id')